## Subhra Kanti Ghosh: Battle of the Neighbourhoods

#### Imports

In [1]:
# Scrape tabular data from wikipedia
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [12]:
#!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge bs4 --yes
!conda install -c conda-forge lxml --yes
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bs4-4.9.1                  |                0           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:           4 KB

The following NEW packages will be INSTALLED:

  bs4                conda-forge/noarch::bs4-4.9.1-0



bs4-4.9.1            | 4 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.



#### Scrape Wiki data

In [61]:
#Scrape wiki using beautiful soup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_largest_cities').text

soup = BeautifulSoup(website_url)
soup.prettify()
My_table = soup.find('table',{'class':'sortable wikitable mw-datatable'}) #putting the data into a table based on the <table> tag
#print('Hi')

### FourSquare Ids

In [103]:
CLIENT_ID = '3Q4RUVQHKBWVWCRIEP30U0Y41KNPCXTPZ2OP1BCMBVSCFEEG' # your Foursquare ID
CLIENT_SECRET = 'TTQRPNFXMBWRRQO2VQSQ4ALXO0ORPAJ54HSHAWRY55YJ3K2J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Create the dataframe

In [154]:
import re

#Create Dataframe
df=pd.DataFrame(columns=['city', 'country','population','area', 'latitude','longitude']) #initialize an empty dataframe
i=0 
rows = My_table.find_all('tr')

#extraxt data for each row
for row in rows:
    cells = row.find_all('td')
    if len(cells)>1:
        city = cells[0].text.strip()
        country = cells[1].text.strip()
        try:
            x=cells[5].text.index('[')
            population = str(cells[5].text.strip())[:x]
        except:
            population = str(cells[5].text.strip())
        try:
            population_num=float(re.sub(r'[^\w\s.]','',population))
        except:
            population_num=0
        try:
            y=cells[6].text.index('[')
            area = str(cells[6].text.strip())[:y]
        except:
            area = str(cells[6].text.strip())
        try:
            area_num=float(re.sub(r'[^\w\s.]','',area))
        except:
            area_num=0
 #       print(city,',', country, ',', population_num, ',', area_num)
        if (population_num>0.0) & (area_num>0.0):
            address = city+','+country
            geolocator = Nominatim(user_agent="cities")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
           # print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
            df.loc[i] = [city,country,population_num,population_num,latitude, longitude]
            i=i+1

df.head()

,city,country,population,area,latitude,longitude
0,Tokyo,Japan,13515271.0,13515271.0,35.682839,139.759455
1,Delhi,India,16753235.0,16753235.0,28.651718,77.221939
2,Shanghai,China,24183000.0,24183000.0,31.225299,121.489050
3,São Paulo,Brazil,12252023.0,12252023.0,-23.550651,-46.633382
4,Mexico City,Mexico,8918653.0,8918653.0,19.432630,-99.133178


#### Create a map of World with cities superimposed on top

In [155]:
# create map of Toronto using latitude and longitude values
map_world = folium.Map(location=[0, 0], zoom_start=2)

# add markers to map
for lat, lng, city, country in zip(df['latitude'], df['longitude'], df['city'], df['country']):
    label = '{}, {}'.format(city, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_world)  
    
map_world

In [156]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### function to repeat the same process to all the neighborhoods in Toronto

In [157]:
def getNearbyVenues(cities, latitudes, longitudes, radius=5000, LIMIT=100):
    ven_list=pd.DataFrame(columns=['city', 'latitude','longitude','name','lat','lng','categories'])
    c=0
    for city, latitude,longitude in zip(cities, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
        
        res = requests.get(url).json()['response']['venues']
        nearby_ven = json_normalize(res)
        # filter columns
        filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
        nearby_ven =nearby_ven.loc[:, filtered_columns]
        
        # filter the category for each row
        nearby_ven['venue.Categories'] = nearby_ven.apply(get_category_type, axis=1)
        
        # clean columns
        nearby_ven.columns = [col.split(".")[-1] for col in nearby_ven.columns]
        nearby_ven.drop(['categories'], axis=1,inplace=True)
        nearby_ven['categories']=nearby_ven['Categories']
        nearby_ven.drop(['Categories'], axis=1,inplace=True)
        nearby_ven['city']=city
        nearby_ven['latitude']=latitude
        nearby_ven['longitude']=longitude
        temp=nearby_ven[['city', 'latitude','longitude','name','lat','lng','categories']]
        ven_list=ven_list.append(temp)
        
    return (ven_list)


In [163]:
city_venues = getNearbyVenues(cities=df['city'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )
city_venues.shape

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(6971, 7)

In [164]:
city_venues.head()

,city,latitude,longitude,name,lat,lng,categories
0,Tokyo,35.682839,139.759455,Imperial Palace (皇居),35.682500,139.752100,Palace
1,Tokyo,35.682839,139.759455,Kokyo Gaien (皇居外苑),35.679928,139.758562,Garden
2,Tokyo,35.682839,139.759455,Wadakura Fountain Park (和田倉噴水公園),35.683453,139.760829,Park
3,Tokyo,35.682839,139.759455,Kikyomon Gate (桔梗門 (内桜田門)),35.684016,139.758522,Historic Site
4,Tokyo,35.682839,139.759455,皇居正門石橋,35.680220,139.754822,Bridge


## Analyze Neighborhood of Cities

In [165]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['city'] = city_venues['city'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,city,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Gate,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Anhui Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Ash and Haleem Place,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Awadhi Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Casino,Castle,Cemetery,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chiropractor,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention,Convention Center,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Courthouse,Coworking Space,Credit Union,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Distribution Center,Dive Shop,Doctor's Office,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Huaiyang Restaurant,Hubei Restaurant,Hunan Restaurant,Hungarian Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,I

In [166]:
city_onehot.shape

(6971, 549)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [167]:
city_grouped = city_onehot.groupby('city').mean().reset_index()
#world_grouped = df.join(world_grouped.set_index('city'), on = 'city')
city_grouped.head()

,city,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Gate,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Anhui Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Ash and Haleem Place,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Awadhi Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Casino,Castle,Cemetery,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chiropractor,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention,Convention Center,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Courthouse,Coworking Space,Credit Union,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Distribution Center,Dive Shop,Doctor's Office,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Huaiyang Restaurant,Hubei Restaurant,Hunan Restaurant,Hungarian Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,I

In [168]:
city_grouped.shape

(74, 549)

### Print each city along with the top 5 most common venues

In [169]:
num_top_venues = 5

for city in city_grouped['city']:
    print("----"+city+"----")
    temp = city_grouped[city_grouped['city'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ahmedabad----
                 venue  freq
0    Indian Restaurant  0.12
1                Hotel  0.06
2        Historic Site  0.04
3                River  0.03
4  Government Building  0.03


----Atlanta----
                 venue  freq
0            City Hall  0.17
1  Government Building  0.13
2           Courthouse  0.11
3     Capitol Building  0.06
4               Church  0.05


----Baghdad----
                       venue  freq
0                       Café  0.08
1                     Office  0.07
2  Middle Eastern Restaurant  0.06
3        Government Building  0.04
4       Fast Food Restaurant  0.04


----Bangalore----
                 venue  freq
0          Event Space  0.10
1  Government Building  0.10
2           Courthouse  0.06
3            City Hall  0.06
4                Hotel  0.04


----Bangkok----
             venue  freq
0       Courthouse  0.07
1      Bus Station  0.05
2          Library  0.04
3  Thai Restaurant  0.04
4     Noodle House  0.02


----Barcelona----
      

In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [171]:
num_top_venues = 5

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    columns.append('Venue No. {}'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['city'] = city_grouped['city']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,city,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
0,Ahmedabad,Indian Restaurant,Hotel,Historic Site,Fast Food Restaurant,Bank
1,Atlanta,City Hall,Government Building,Courthouse,Capitol Building,Church
2,Baghdad,Café,Office,Middle Eastern Restaurant,Government Building,Fast Food Restaurant
3,Bangalore,Event Space,Government Building,Courthouse,City Hall,Fast Food Restaurant
4,Bangkok,Courthouse,Bus Station,Library,Thai Restaurant,University


## Cluster Cities

In [172]:
city_grouped = df.join(city_grouped.set_index('city'), on = 'city')
city_grouped.head()

,city,country,population,area,latitude,longitude,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Gate,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Anhui Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Ash and Haleem Place,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Awadhi Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Casino,Castle,Cemetery,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chiropractor,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention,Convention Center,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Courthouse,Coworking Space,Credit Union,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Distribution Center,Dive Shop,Doctor's Office,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Huaiyang Restaurant,Hubei Restaurant,Hunan Restaurant,Hungarian Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indonesian Rest

In [173]:
# set number of clusters
kclusters = 5

city_grouped_clustering = city_grouped.drop(['city','country','latitude','longitude'], 1)
city_grouped_clustering.head()
# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 

,population,area,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport Gate,Airport Service,Alternative Healer,American Restaurant,Amphitheater,Anhui Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Ash and Haleem Place,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,Awadhi Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Campaign Office,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Casino,Castle,Cemetery,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chiropractor,Church,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention,Convention Center,Corporate Amenity,Corporate Cafeteria,Cosmetics Shop,Courthouse,Coworking Space,Credit Union,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Distribution Center,Dive Shop,Doctor's Office,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fire Station,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Funeral Home,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Huaiyang Restaurant,Hubei Restaurant,Hunan Restaurant,Hungarian Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Industri

In [176]:
from sklearn.preprocessing import StandardScaler
cluster_dataset = StandardScaler().fit_transform(city_grouped_clustering)
#cluster_dataset

k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 0 0 1 0 0 0 1 0 3 1 0 2 0 4 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 0]


In [177]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [178]:
city_merged = df

city_merged = city_merged.join(city_venues_sorted.set_index('city'), on='city')

city_merged.head() # check the last columns!

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
0,Tokyo,Japan,13515271.0,13515271.0,35.682839,139.759455,1,Historic Site,Office,Japanese Restaurant,Convenience Store,Lounge
1,Delhi,India,16753235.0,16753235.0,28.651718,77.221939,0,Indian Restaurant,Dessert Shop,Bank,Snack Place,Office
2,Shanghai,China,24183000.0,24183000.0,31.225299,121.489050,2,Chinese Restaurant,Asian Restaurant,Shanghai Restaurant,Ice Cream Shop,Noodle House
3,São Paulo,Brazil,12252023.0,12252023.0,-23.550651,-46.633382,1,Cosmetics Shop,Historic Site,Exhibit,Street Art,Business Service
4,Mexico City,Mexico,8918653.0,8918653.0,19.432630,-99.133178,1,Other Event,Mexican Restaurant,Historic Site,Plaza,Festival


In [181]:
# create map
map_clusters = folium.Map(location=[0, 0], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['latitude'], city_merged['longitude'], city_merged['city'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 0

In [182]:
city_merged.loc[city_merged['Cluster Labels'] == 0]

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
1,Delhi,India,16753235.0,16753235.0,28.651718,77.221939,0,Indian Restaurant,Dessert Shop,Bank,Snack Place,Office
5,Cairo,Egypt,9500000.0,9500000.0,30.048819,31.243666,0,Café,Bank,Bookstore,Hotel,Office
6,Mumbai,India,12478447.0,12478447.0,18.938771,72.835335,0,Fast Food Restaurant,Bar,Indian Restaurant,Restaurant,Sandwich Place
8,Dhaka,Bangladesh,14399000.0,14399000.0,23.759357,90.378814,0,Restaurant,Office,Fast Food Restaurant,Arts & Crafts Store,Bank
10,New York City,United States,8398748.0,8398748.0,40.712728,-74.006015,0,Office,Bus Station,Lawyer,Plaza,Food Truck
12,Buenos Aires,Argentina,3054300.0,3054300.0,-34.607568,-58.437089,0,Argentinian Restaurant,Automotive Shop,Bus Stop,Concert Hall,Pizza Place
14,Istanbul,Turkey,15029231.0,15029231.0,41.076602,29.052495,0,Boat or Ferry,Cruise,Harbor / Marina,General Entertainment,Pier
15,Kolkata,India,4496694.0,4496694.0,22.545412,88.356775,0,Salon / Barbershop,Residential Building (Apartment / Condo),Café,Office,Bakery
17,Rio de Janeiro,Brazil,6520000.0,6520000.0,-22.911014,-43.209373,0,Convention Center,Event Space,Office,Fair,Residential Building (Apartment / Condo)
19,Kinshasa,DR Congo,11462000.0,11462000.0,-4.321706,15.312597,0,Bar,Bank,Office,Mobile Phone Shop,Comedy Club


#### Cluster 1

In [183]:
city_merged.loc[city_merged['Cluster Labels'] == 1]

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
0,Tokyo,Japan,13515271.0,13515271.0,35.682839,139.759455,1,Historic Site,Office,Japanese Restaurant,Convenience Store,Lounge
3,São Paulo,Brazil,12252023.0,12252023.0,-23.550651,-46.633382,1,Cosmetics Shop,Historic Site,Exhibit,Street Art,Business Service
4,Mexico City,Mexico,8918653.0,8918653.0,19.432630,-99.133178,1,Other Event,Mexican Restaurant,Historic Site,Plaza,Festival
7,Beijing,China,21707000.0,21707000.0,39.906217,116.391276,1,Historic Site,Hotel,Fast Food Restaurant,Government Building,Chinese Restaurant
22,Moscow,Russia,13200000.0,13200000.0,55.750446,37.617494,1,Monument / Landmark,History Museum,Plaza,Historic Site,Exhibit
33,Nagoya,Japan,2320361.0,2320361.0,35.185105,136.899844,1,Historic Site,Castle,Souvenir Shop,Monument / Landmark,Japanese Restaurant
43,Ahmedabad,India,5570585.0,5570585.0,23.021624,72.579707,1,Indian Restaurant,Hotel,Historic Site,Fast Food Restaurant,Bank
60,Dallas,United States,1345047.0,1345047.0,32.776272,-96.796856,1,Government Building,Building,Parking,Residential Building (Apartment / Condo),Food Truck
70,Barcelona,Spain,1620343.0,1620343.0,41.382894,2.177432,1,Hotel,Ice Cream Shop,Residential Building (Apartment / Condo),Gift Shop,City Hall
71,"Washington, D.C.",United States,702455.0,702455.0,38.894985,-77.036571,1,Government Building,Monument / Landmark,Office,Food Truck,Library


#### Cluster 2

In [184]:
city_merged.loc[city_merged['Cluster Labels'] == 2]

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
2,Shanghai,China,24183000.0,24183000.0,31.225299,121.489050,2,Chinese Restaurant,Asian Restaurant,Shanghai Restaurant,Ice Cream Shop,Noodle House
9,Osaka,Japan,2725006.0,2725006.0,34.619881,135.490357,2,Japanese Restaurant,Convenience Store,Chinese Restaurant,Café,Salon / Barbershop
13,Chongqing,China,30165500.0,30165500.0,29.558571,106.549282,2,Chinese Restaurant,Hotel,Noodle House,Bus Station,Convenience Store
18,Tianjin,China,15569000.0,15569000.0,39.123564,117.198078,2,Chinese Restaurant,Noodle House,Japanese Restaurant,Coffee Shop,BBQ Joint
20,Guangzhou,China,14498400.0,14498400.0,23.130196,113.259294,2,Chinese Restaurant,Fast Food Restaurant,Japanese Restaurant,Szechuan Restaurant,Bus Station
23,Shenzhen,China,12528300.0,12528300.0,22.555454,114.054330,2,Café,Coffee Shop,Bakery,Fast Food Restaurant,Chinese Restaurant
39,Nanjing,China,7260000.0,7260000.0,32.060974,118.791646,2,Chinese Restaurant,Building,Historic Site,College Administrative Building,University
40,Wuhan,China,10892900.0,10892900.0,30.595105,114.299935,2,Chinese Restaurant,Bar,Coffee Shop,Café,Residential Building (Apartment / Condo)
45,Xi'an,China,8989000.0,8989000.0,34.346842,108.936589,2,Chinese Restaurant,Coffee Shop,Shopping Mall,Building,Hotpot Restaurant
47,Dongguan,China,8342500.0,8342500.0,23.044471,113.746551,2,Chinese Restaurant,Dessert Shop,Café,Hotel,Fast Food Restaurant


#### Cluster 3

In [185]:
city_merged.loc[city_merged['Cluster Labels'] == 3]

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
11,Karachi,Pakistan,14910352.0,14910352.0,25.14469,67.184777,3,Farm,Resort,General Entertainment,Harbor / Marina,Housing Development


#### Cluster 4

In [186]:
city_merged.loc[city_merged['Cluster Labels'] == 4]

,city,country,population,area,latitude,longitude,Cluster Labels,Venue No. 1,Venue No. 2,Venue No. 3,Venue No. 4,Venue No. 5
16,Manila,Philippines,1780148.0,1780148.0,14.590622,120.979970,4,College Classroom,College Administrative Building,City Hall,Historic Site,College Lab
21,Los Angeles,United States,3990456.0,3990456.0,34.053691,-118.242767,4,City Hall,Government Building,Courthouse,Moving Target,Office
25,Bangalore,India,8443675.0,8443675.0,12.979120,77.591300,4,Event Space,Government Building,Courthouse,City Hall,Fast Food Restaurant
44,Kuala Lumpur,Malaysia,1768000.0,1768000.0,3.151696,101.694237,4,City Hall,Office,Building,Café,Government Building
61,Toronto,Canada,2731571.0,2731571.0,43.653482,-79.383935,4,Office,General Entertainment,City Hall,Music Venue,Coffee Shop
66,Philadelphia,United States,1526006.0,1526006.0,39.952724,-75.163526,4,City Hall,Monument / Landmark,Bus Line,Building,Metro Station
67,Atlanta,United States,420003.0,420003.0,33.749099,-84.390185,4,City Hall,Government Building,Courthouse,Capitol Building,Church
